In [1]:
import boto3
import sagemaker
from sagemaker import get_execution_role
# from sagemaker.image_uris import retrieve
from sagemaker.s3 import *
import sys

if int(sagemaker.__version__.split('.')[0]) == 2:
    !{sys.executable} -m pip install sagemaker==1.72.0
    print("Installing previous SageMaker Version. Please restart the kernel")
else:
    print("Version is good")

role = get_execution_role()
sess = sagemaker.Session(default_bucket=None)
region = boto3.session.Session().region_name
print("Region = {}".format(region))
sm = boto3.Session().client('sagemaker')


Version is good
Region = eu-central-1


In [2]:

# see https://sagemaker.readthedocs.io/en/stable/api/utility/image_uris.html for which inputs to use
# see https://github.com/aws/deep-learning-containers/blob/master/available_images.md for registry paths with custom algorithms


training_image = "763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:1.6.0-gpu-py36-cu101-ubuntu16.04"
print (training_image)

763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:1.6.0-gpu-py36-cu101-ubuntu16.04


In [4]:
!pip install sagemaker-experiments 
from sagemaker.analytics import ExperimentAnalytics
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent
from smexperiments.tracker import Tracker

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.


ContextualVersionConflict: (boto3 1.16.19 (/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages), Requirement.parse('boto3>=1.16.27'), {'sagemaker-experiments'})

In [10]:
rawbucket= sess.default_bucket() # Alternatively you can use our custom bucket here. 
data_bucket = "treetracker-training-images"
prefix = 'sagemaker-modelmonitor' # use this prefix to store all files pertaining to this workshop.

dataprefix = prefix + '/data'
traindataprefix = prefix + '/train_data'
testdataprefix = prefix + '/test_data'
testdatanolabelprefix = prefix + '/test_data_no_label'
trainheaderprefix = prefix + '/train_headers'

s3_train_key = "imnet"
s3_validation_key = ""
s3_train = 's3://{}/{}/'.format(data_bucket, s3_train_key)
s3_validation = 's3://{}/{}/'.format(data_bucket, s3_validation_key)
sagemaker_train = 's3://{}/{}/'.format(rawbucket, s3_train_key)
sagemaker_validation = 's3://{}/{}/'.format(rawbucket, s3_validation_key)


In [11]:
train_data_location = rawbucket + '/' + traindataprefix
test_data_location = rawbucket+'/'+testdataprefix
print("Training data location = {}".format(train_data_location))
print("Test data location = {}".format(test_data_location))

Training data location = sagemaker-eu-central-1-053061259712/sagemaker-modelmonitor/train_data
Test data location = sagemaker-eu-central-1-053061259712/sagemaker-modelmonitor/test_data


In [12]:

from sagemaker.processing import ProcessingInput, ProcessingOutput

pre_input = ProcessingInput(source=s3_train, destination=None, input_name="pre")
pre_output = ProcessingOutput(source=None, destination=sagemaker_train)



In [13]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point='mobilenet_v2.py',
                    role=role,
                    framework_version='1.4.0',
                    train_instance_count=2,
                    train_instance_type='ml.c4.xlarge',
                    hyperparameters={
                        'epochs': 6,
                        'backend': 'gloo'
                    })

In [ ]:
estimator.fit({"training": s3_train})

'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


2020-12-10 04:03:09 Starting - Starting the training job...
2020-12-10 04:03:11 Starting - Launching requested ML instances......
2020-12-10 04:04:17 Starting - Preparing the instances for training......
2020-12-10 04:05:23 Downloading - Downloading input data...